In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from utils.nn.sequential import Sequential, EarlyStopping
from utils.nn.layer import Dense
from utils.nn.neuron import Activation
from utils.nn.optimizer import Adam
from utils.nn.losses import get_loss_function
from utils.nn.initializer import Initializer, InitializationType

# Universal Function Approximation

## Approximate Function 1

**Let's use Multilayer Perceptron to learn approximation for this function on interval $[0, 15]$**

$f(x) = -0.03 \cdot x^2 \cdot 0.05 \cdot \ln(x^3) + 0.5 \cdot \cos\left(\frac{x}{1.5}\right) + 0.1 \cdot \sin(5x)$


In [ ]:
def complicated_function(x):
    return -0.03 * x**2  * 0.05 * np.log(x ** 3) + 0.5 * np.cos(x/1.5) + 0.1 * np.sin(x*5)

def build_and_train_approximator():
    """Builds, trains, and evaluates a function approximator model."""

    # 1. Generate training data
    x_values = np.linspace(0.1, 15, 200).reshape(-1, 1)
    noise = np.random.normal(0, 0.05, x_values.shape)
    y_values = complicated_function(x_values) + noise
    
    # 2. Define the model architecture
    # FIX: Pass the shape tuple directly to the Dense layer.
    # The Dense layer will correctly pass these dimensions to the Initializer.
    model = Sequential([
        Dense(shape=(1, 64), activation=Activation.RELU, initializer=Initializer(InitializationType.HE_UNIFORM, fan_in=1, fan_out=64)),
        Dense(shape=(64, 64), activation=Activation.RELU, initializer=Initializer(InitializationType.HE_UNIFORM, fan_in=64, fan_out=64)),
        Dense(shape=(64, 1), activation=Activation.LINEAR, initializer=Initializer(InitializationType.GLOROT_UNIFORM, fan_in=64, fan_out=1))
    ])

    # Print a summary of the model
    model.summary()

    # 3. Compile and train the model
    optimizer = Adam(params=model.parameters(), learning_rate=0.001)
    loss_function = get_loss_function('mse')
    
    early_stopping = EarlyStopping(patience=20, min_delta=0.0001, restore_best_weights=True)

    model.fit(
        x_train=x_values,
        y_train=y_values,
        optimizer=optimizer,
        loss_func=loss_function,
        epochs=1000,
        batch_size=32,
        metric="r2_score",
        early_stopping=early_stopping,
        display_interval=50
    )

    # 4. Evaluate and visualize the approximation
    x_test = np.linspace(0.1, 15, 1000).reshape(-1, 1)
    y_true = complicated_function(x_test)
    y_pred = model.predict(x_test)
    
    r2 = model.evaluate(x_test, y_true, metric_name="r2_score")[1]
    print(f"\nR2 Score on test data: {r2:.4f}")
    
    # Visualization using Seaborn and Matplotlib
    plt.style.use('seaborn-v0_8-darkgrid')
    plt.figure(figsize=(12, 8))

    # Plot the original true function
    sns.lineplot(x=x_test.flatten(), y=y_true.flatten(), color='blue', linewidth=3, label='True Function', zorder=10)

    # Plot the noisy data points as a scatter plot
    sns.scatterplot(x=x_values.flatten(), y=y_values.flatten(), color='red', alpha=0.5, label='Noisy Training Data', zorder=1)

    # Plot the model's predictions
    sns.lineplot(x=x_test.flatten(), y=y_pred.flatten(), color='green', linewidth=3, linestyle='--', label='MLP Approximation', zorder=5)

    plt.title('Function Approximation using a Multilayer Perceptron', fontsize=18)
    plt.xlabel('x', fontsize=14)
    plt.ylabel('y', fontsize=14)
    plt.legend(fontsize=12)
    plt.show()
    
    return x_test, y_true, y_pred

build_and_train_approximator()

## Approximate Function 2


In [ ]:
def dampened_wave_function_2(x):
    """
    A function that represents a dampened wave.
    """
    return np.exp(-0.5 * x) * np.sin(x * 2)

x_values_2 = np.linspace(0, 10, 200)  # Using more points for a smoother curve
y_values_2 = dampened_wave_function_2(x_values_2)

model_2 = Sequential(
    layers=[
        Dense(
            shape=(1, 32),  # Increased neurons for more complexity
            activation=Activation.TANH,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        ),
        Dense(
            shape=(32, 16),
            activation=Activation.TANH,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        ),
        Dense(
            shape=(16, 16),
            activation=Activation.RELU,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        ),
        Dense(
            shape=(16, 1),
            activation=Activation.LINEAR,
            initializer=Initializer(fill_type=InitializationType.RANDOM_NORMAL)
        )
    ]
)

approximator2 = FuncApproximator(
    model=model_2,
    scalar_func=dampened_wave_function_2,
    x=x_values_2,
    y=y_values_2
)

approximator2.fit(
    batch_size=8,
    test_split_size=0.15,
    epochs=40,  # Increased epochs for better fitting
    display_on_each_n_step=1,
    learning_rate=0.005
)

approximator2.visualize_fit()